In [86]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import cudf
import cuml
%matplotlib inline

### <center>Data Dictionary</center> ###
1. survival --> Survival 	0 = No, 1 = Yes<br/>
2. pclass --> Ticket class 	1 = 1st, 2 = 2nd, 3 = 3rd

In [87]:
train = pd.read_csv('test.csv')

In [88]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [89]:
train.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [90]:
train['Age'].isna().any() #there is nan values

True

### <center>What to Say: Miss, Ms. or Mrs., Mr. and Master</center> ###
<a href="https://dosaygive.com/miss-ms-mrs/"><u>https://dosaygive.com/miss-ms-mrs/</u></a><br/>
1. <b>Miss</b> – Some still use it for any unmarried woman, mainly it is for girls 18 years old and younger.<br/>
2. <b>Mrs.</b> – For married women. Also acceptable for divorced and widowed women.<br/>
3. <b>Ms.</b> – Can be used for any woman over the age of 18.<br/>
4. <b>Mr</b> For men 18 and over, For boys under the age of 18, no title needed.<br/>
5. <b>Master</b> is used for for boy under the age of 8 and only in formal invitations.<br/>

In [91]:
missing_Age = train[train['Age'].isna()].loc[:, ['Name', 'Age']]

In [92]:
len(missing_Age) #there is 177 data whith missing age

86

In [93]:
len(missing_Age) + len(train[~train['Age'].isna()]) #verify the sum of data

418

In [94]:
type(missing_Age)

pandas.core.frame.DataFrame

In [95]:
missing_Age.head()

,Name,Age
10,"Ilieff, Mr. Ylio",NaN
22,"Flegenheim, Mrs. Alfred (Antoinette)",NaN
29,"Samaan, Mr. Elias",NaN
33,"Johnston, Mrs. Andrew G (Elizabeth Lily"" Watson)""",NaN
36,"Roth, Miss. Sarah A",NaN


In [96]:
missing_Age.index

Int64Index([ 10,  22,  29,  33,  36,  39,  41,  47,  54,  58,  65,  76,  83,
             84,  85,  88,  91,  93, 102, 107, 108, 111, 116, 121, 124, 127,
            132, 133, 146, 148, 151, 160, 163, 168, 170, 173, 183, 188, 191,
            199, 200, 205, 211, 216, 219, 225, 227, 233, 243, 244, 249, 255,
            256, 265, 266, 267, 268, 271, 273, 274, 282, 286, 288, 289, 290,
            292, 297, 301, 304, 312, 332, 339, 342, 344, 357, 358, 365, 366,
            380, 382, 384, 408, 410, 413, 416, 417],
           dtype='int64')

In [97]:
target_index = missing_Age.index

In [98]:
type(target_index)

pandas.core.indexes.numeric.Int64Index

In [99]:
train['Age'].describe()

count    332.000000
mean      30.272590
std       14.181209
min        0.170000
25%       21.000000
50%       27.000000
75%       39.000000
max       76.000000
Name: Age, dtype: float64

In [100]:
missing_Age.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86 entries, 10 to 417
Data columns (total 2 columns):
Name    86 non-null object
Age     0 non-null float64
dtypes: float64(1), object(1)
memory usage: 2.0+ KB


In [101]:
target_index[0]

10

In [102]:
index_length = range(len(target_index) - 1)

In [103]:
def change_women_age(x):
    for i in target_index:
        if 'Mrs' in x.loc[i, 'Name']:
            x.loc[i, ['Age']] = np.random.randint(19, 81, 1)[0]
        if 'Miss' in x.loc[i, 'Name']:
            x.loc[i, ['Age']] = np.random.randint(2, 19, 1)[0]
        if 'Ms' in x.loc[i, 'Name']:
            x.loc[i, ['Age']] = np.random.randint(19, 30, 1)[0]
        else:
            pass

In [104]:
change_women_age(missing_Age)

In [105]:
train['Age'].describe()

count    332.000000
mean      30.272590
std       14.181209
min        0.170000
25%       21.000000
50%       27.000000
75%       39.000000
max       76.000000
Name: Age, dtype: float64

In [106]:
missing_Age

,Name,Age
10,"Ilieff, Mr. Ylio",NaN
22,"Flegenheim, Mrs. Alfred (Antoinette)",39.0
29,"Samaan, Mr. Elias",NaN
33,"Johnston, Mrs. Andrew G (Elizabeth Lily"" Watson)""",53.0
36,"Roth, Miss. Sarah A",4.0
39,"Hee, Mr. Ling",NaN
41,"Franklin, Mr. Thomas Parham",NaN
47,"Shaughnessy, Mr. Patrick",NaN
54,"Mangiavacchi, Mr. Serafino Emilio",NaN
58,"Davison, Mr. Thomas Henry",NaN


In [107]:
import re #avoiding colliding Mr. and Mrs.
regex = re.compile('Mr\W')
def change_men_age(x):
    for i in target_index:
        if regex.findall(x.loc[i, 'Name']):
            x.loc[i, ['Age']] = np.random.randint(18, 30, 1)[0]
        if 'Master' in x.loc[i, 'Name']:
            x.loc[i, ['Age']] = np.random.randint(2, 8, 1)[0]
        else:
            pass

In [108]:
change_men_age(missing_Age)

In [109]:
missing_Age

,Name,Age
10,"Ilieff, Mr. Ylio",24.0
22,"Flegenheim, Mrs. Alfred (Antoinette)",39.0
29,"Samaan, Mr. Elias",21.0
33,"Johnston, Mrs. Andrew G (Elizabeth Lily"" Watson)""",53.0
36,"Roth, Miss. Sarah A",4.0
39,"Hee, Mr. Ling",27.0
41,"Franklin, Mr. Thomas Parham",19.0
47,"Shaughnessy, Mr. Patrick",18.0
54,"Mangiavacchi, Mr. Serafino Emilio",27.0
58,"Davison, Mr. Thomas Henry",20.0


In [110]:
missing_Age.isna().any() #there is still Nan value

Name    False
Age     False
dtype: bool

In [111]:
mask_age = missing_Age['Age'].isna()

In [112]:
missing_Age.loc[mask_age] #check the value of Nan

,Name,Age


In [113]:
missing_Age.loc[766, 'Age'] = np.random.randint(45, 81, 1)[0] #Doctoral Age

In [114]:
missing_Age.isna().any() #there is no nan value

Name     True
Age     False
dtype: bool

In [115]:
mask_age = train.loc[:, 'Age'].isna()

In [116]:
train.loc[mask_age, 'Age']

10    NaN
22    NaN
29    NaN
33    NaN
36    NaN
       ..
408   NaN
410   NaN
413   NaN
416   NaN
417   NaN
Name: Age, Length: 86, dtype: float64

In [117]:
train.loc[mask_age, 'Age'] = missing_Age.loc[:, 'Age'] #replace with values that we have set before

In [118]:
train.loc[:, 'Age'].isna().any()

False

In [119]:
train.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [120]:
train.info() #there is still nan value in Cabin and embarked

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            418 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [121]:
train['Cabin'].value_counts(dropna=False) #examine the values of Cabin

NaN                327
B57 B59 B63 B66      3
C101                 2
C116                 2
C78                  2
                  ... 
B26                  1
F                    1
A29                  1
D40                  1
C54                  1
Name: Cabin, Length: 77, dtype: int64

In [122]:
train['Cabin'].fillna('No Cabin', inplace=True) #change with No Cabin

In [123]:
train.info() #there is nan value in embarked

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            418 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          418 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [124]:
train['Embarked'].fillna('No Embarked', inplace=True)

In [125]:
train['Embarked'].value_counts()

S    270
C    102
Q     46
Name: Embarked, dtype: int64

In [126]:
train.info() #there is still Nan value in Fare

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            418 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          418 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [127]:
train['Fare'].fillna(train['Fare'].mean(), inplace=True)

In [128]:
train.info() #lets save to CSV

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            418 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           418 non-null float64
Cabin          418 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [129]:
train.to_csv('test_noNAN.csv')